# Income Classification Using Machine Learning

This project uses the Adult Income dataset to predict whether a person earns more than 50K per year based on various demographic attributes. 

The steps include:
- Data loading and exploration
- Data preprocessing
- Handling missing values
- Feature transformation
- Handling class imbalance using SMOTE
- Building and tuning models (LightGBM, GridSearchCV)
- Evaluation using classification metrics


In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

# Load the Dataset

In [ ]:
df = pd.read_csv("adult income1.csv")

# Initial Exploration (df.head(), df.info(), df.describe())

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

# Missing Values + Data Cleaning

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df['income'].unique()

In [ ]:
df['TARGET'] = df['income'].apply(lambda x: 1 if x == '>50K' else 0)

In [ ]:
df.head(5)

In [ ]:
df['occupation'].value_counts()

In [ ]:
df['occupation'] = df['occupation'].replace('?', np.nan)

#  EDA (Exploratory Data Analysis)

In [ ]:
df['TARGET'].value_counts()

In [ ]:
# Income Distribution

plt.pie(df['TARGET'].value_counts(), explode=[0,0.1], wedgeprops={'edgecolor':'k', 'width':0.9}, shadow=True, colors=['green', 'yellow'],
        autopct='%1.1f%%', startangle=30)
plt.legend(loc='upper left', labels=['Below 50', 'Above 50'])
plt.show()

In [ ]:
df['sex'].value_counts()

In [ ]:
# Gender Distribution

plt.pie(df['sex'].value_counts(), explode=[0,0.1], wedgeprops={'edgecolor':'k', 'width':0.9}, shadow=True, colors=['red', 'orange'],
        autopct='%1.1f%%', startangle=50)
plt.legend(loc='upper left', labels=['Male', 'Female'])
plt.show()

In [ ]:
sns.heatmap(df.corr(numeric_only=True), cmap='coolwarm')
plt.show()

In [ ]:
sns.pairplot(df[['age', 'education.num', 'fnlwgt', 'TARGET']], hue='TARGET')

In [ ]:
df.head(5)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore')

In [ ]:
sm = SimpleImputer(strategy='most_frequent')

In [ ]:
X = df.drop(columns={'education', 'fnlwgt', 'TARGET', 'income'})

In [ ]:
y = df['TARGET']

# Data Preprocessing (Pipeline)

In [ ]:
from sklearn.pipeline import Pipeline 

In [ ]:
pipeline = Pipeline([
    ('imputer', sm),
    ('encoder', ohe),
])  

In [ ]:
from sklearn.compose import make_column_transformer

In [ ]:
columns = ['workclass', 'occupation', 'relationship', 'native.country', 'sex', 'race', 'marital.status']

In [ ]:
# Column Transformer

col_transformer = make_column_transformer(
    (pipeline, columns),
    remainder='passthrough'
)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

# Model Setup: LightGBM

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgbm = LGBMClassifier(random_state=42, class_weight='balanced', verbosity=-1)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smt = SMOTE(random_state=42)

# Pipeline + SMOTE

In [ ]:
from imblearn.pipeline import Pipeline as imb_pipe

In [ ]:
pipe = imb_pipe([
    ('preprocessor', col_transformer),
    ('smote', smt),
    ('classifier', lgbm)
])

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Model Tuning with GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [10, 15, 20],
    'classifier__learning_rate': [0.05, 0.1],
    'classifier__num_leaves': [30, 40, 50],
    'classifier__class_weight': ['balanced'],
    'classifier__boosting_type': ['gbdt'],
}

In [ ]:
GSCV = GridSearchCV(pipe, param_grid, cv=skf, scoring='f1')

In [ ]:
GSCV.fit(X_train, y_train)

In [ ]:
GSCV_pre = GSCV.predict(X_test)

In [ ]:
GSCV.best_params_

# Model Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
cm = confusion_matrix(GSCV_pre, y_test)

In [ ]:
print(cm)

In [ ]:
asc = accuracy_score(GSCV_pre, y_test)

In [ ]:
print(round(asc, 2))

In [ ]:
cr = classification_report(GSCV_pre, y_test)

In [ ]:
print(cr)